In [5]:
# Imports
import mne
import numpy as np
import matplotlib.pyplot as plt
import importlib
importlib.import_module('mpl_toolkits.axes_grid').__path__
import nelpy as nel
import nelpy.plotting as npl
import scipy.io
from scipy import signal
from copy import *
import pandas as pd
from ghost.wave import ContinuousWaveletTransform
import ghost.wave as gwave
from seegpreprocessing import *
import ghostipy as gsp

import warnings
warnings.filterwarnings('ignore')

import holoviews as hv
import hvplot
from holoviews import opts
hv.extension('bokeh', 'matplotlib')

import csv


ModuleNotFoundError: No module named 'mpl_toolkits.axes_grid'

In [2]:
%matplotlib qt

# Loading the Data

If grabbing signal from `data`, note that `data[channel]` will return a tuple, where the array at index `0` contains the values of the signal, and the array at index `1` contains the timestamps.

In [20]:
# Loading SEEG Data
path = "C:\\Users\\amand\\Documents\\Research\\Project_AHN\\Data_processing\\data\\meditation_sleep\\p3\\M3_Sleep.edf"
# path = '/home/ariel/Documents/CMU/AHN/test4~ cmu_5c0d9c08-868c-458a-87e8-6a17ffc46069.edf'
data = mne.io.read_raw_edf(path, preload=True)
raw_data = data.get_data()

# Loading corresponding .mat file
# pat_csv = pd.read_csv('/home/ariel/Documents/CMU/AHN/p3_08162023/behavior/p2_stroop1_july17.csv')

Extracting EDF parameters from /home/ariel/Documents/CMU/AHN/test4~ cmu_5c0d9c08-868c-458a-87e8-6a17ffc46069.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 289791  =      0.000 ...   282.999 secs...


In [21]:
ant_ins_asa = nel.AnalogSignalArray(raw_data[0:4,:], fs=1024)

# Extracting trigger events and metadata
Trigger mapping:
1. Start trial
2. Word display
3. Button press

In [22]:
1542.375/60

25.70625

In [10]:
trig_asa.data.shape

(1, 151936)

In [17]:
# you can get the metadata included in the file and a list of all channels:
info = data.info
channels = data.ch_names
trig_idx = np.where(np.array(channels) == 'TRIG')[0][0]
trig_asa = nel.AnalogSignalArray(raw_data[trig_idx,:], fs=1024)

In [18]:
# Find event triggers (rising and falling edge)
trigger_events = mne.find_events(data, stim_channel="TRIG",output='onset')
trig_starts = trigger_events[:,0]/1024
trig_ends = trigger_events[:,0] + 102
trig_ends = trig_ends/1024
trigger_epochs = nel.EpochArray(np.vstack((trig_starts, trig_starts+0.1)).T)

1 events found
Event IDs: [248]


In [13]:
overlap = stroop_epochs.intersect(trigger_epochs)

NameError: name 'stroop_epochs' is not defined

In [19]:
npl.plot(trig_asa)
# npl.plot(trig_asa[overlap[1].expand(.1)], color='orange')

In [71]:
# display_epochs = trigger_epochs[np.arange(1,324,6)]
display_epochs = overlap[np.arange(0,1500,2)]
# display_epochs = stroop_triggers[np.arange(1,4860,6)]
# display_epochs = trigger_epochs[np.hstack((np.arange(1,324,6), np.arange(969,1940,6)))]
# key_press_epochs = trigger_epochs[np.arange(3,324,6)]
key_press_epochs = overlap[np.arange(1,1500,2)]
# key_press_epochs = trigger_epochs[np.hstack((np.arange(3,324,6), np.arange(971,1940,6)))]

In [72]:
npl.plot(trig_asa)
npl.plot(trig_asa[display_epochs.expand(0.001)], color='orange')
npl.plot(trig_asa[key_press_epochs.expand(0.001)], color='green')

In [79]:
key_press_epochs.n_intervals

750

In [78]:
c_idx.shape[0]+nc_idx.shape[0]

750

## Amanda's preprocessing

In [8]:
import seeg_constants as CONST

In [9]:
subject = CONST.Subject('p3')

In [10]:
patient = 'p3'
event = "key"  # On what even to align stroop tasks. Choose "start" (trial start), "stim" (stimulus onset), or "key" (keypress)
alignment = "left"  # Choose "center" or "left"
width = 1000  # Trials will be this long
start_time = 0  # Start measuring width from this timeregion = subject.entorhinal
region = subject.amcc
probes = len(region)
dim = int(np.ceil(np.sqrt(probes)))

In [53]:
csv_path = patient + "_behavior_" + alignment + str(width) + "_" + event + ".csv"
npy_path = patient + "_trials_" + alignment + str(width) + "_" + event + ".npy"
ch_path = patient + "_ch.csv"

In [12]:
# Read in data
behavior_df = pd.read_csv(csv_path)
with open(npy_path, 'rb') as f:
    trials = np.load(f)
    trial_times = np.load(f)
channels_raw = next(csv.DictReader(open(ch_path)))
channels = {i: int(j) for i, j in channels_raw.items()}

In [54]:
congruency = behavior_df['congruent'].to_numpy()
c_idx = np.where(congruency == 1)[0]
nc_idx = np.where(congruency == 0)[0]
c_trials = np.mean(trials[c_idx, :, start_time:width], axis=0)
nc_trials = np.mean(trials[nc_idx, :, start_time:width], axis=0)

In [14]:
# Plot trials
c_stderr = np.std(trials[c_idx, :, start_time:width], axis=0) / np.sqrt(len(c_idx))
nc_stderr = np.std(trials[nc_idx, :, start_time:width], axis=0) / np.sqrt(len(nc_idx))
t = np.arange(start_time, width) if alignment == "left" else np.arange(-int(width/2), int(width/2))
for i, c in enumerate(region):
    plt.subplot(dim, dim, i+1)
    plt.errorbar(t, c_trials[channels[c]], yerr=c_stderr[channels[c]], label="congruent", ecolor='#f17386', fmt='k')
    plt.errorbar(t, nc_trials[channels[c]], yerr=nc_stderr[channels[c]], label="incongruent", ecolor='#73a1f1', fmt='b')
    y_max = max(max(c_trials[channels[c]]), max(nc_trials[channels[c]]))
    y_min = min(min(c_trials[channels[c]]), min(nc_trials[channels[c]]))
    plt.plot([0, 0], [y_min, y_max], 'k--')
    # plt.plot(t, c_trials[channels[c]], label="congruent", color='k')
    # plt.plot(t, nc_trials[channels[c]], label="incongruent", color='b')
    if i == 4:
        plt.ylabel("Voltage (uV)")
    plt.title('Channel ' + str(i + 1))
    plt.xticks(fontsize=10, rotation=0)
    plt.yticks(fontsize=10, rotation=0)
    plt.legend(loc='upper right', fontsize='small')
plt.suptitle("Stroop, AmCC (left): 1 second after keypress")
plt.show()

## CWT of congruent vs. incongruent

In [19]:
import seeg_library as slib

In [51]:
tup = slib.get_exp_endpoints(data,18)
stroop_epochs = nel.EpochArray(np.vstack((tup[0:10], tup[12:17]))/1024)
bart_epochs = nel.EpochArray(np.vstack((tup[10:12], tup[17]))/1024)

In [98]:
npl.plot(trig_asa)
npl.plot(trig_asa[stroop_epochs], color='orange')
npl.plot(trig_asa[bart_epochs], color='green')

In [13]:
c_trials[channels[c]].shape

(1000,)

In [147]:
with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    coefs_cwt, _, f_cwt, t_cwt, _ = gsp.cwt(
        # amcc_filtered.signals[0][nel.EpochArray([0,470])].data.squeeze(), fs=1024, freq_limits=[1, 60])
    ant_ins_asa.signals[2].data.squeeze(), fs=1024, freq_limits=[1, 60])

In [148]:
# will be normalized such that max is 1, so the
# sampling rate factor can be dropped
psd_cwt = coefs_cwt.real**2 + coefs_cwt.imag**2

In [149]:
psd_cwt[:,0:(450*1024)].std(axis=1).shape

(60,)

In [150]:
shifted = (psd_cwt - np.reshape(psd_cwt[:,0:(450*1024)].mean(axis=1), (60,1))) / np.reshape(psd_cwt[:,0:(450*1024)].std(axis=1), (60,1))

In [151]:
offset = 1024
ep_power = np.zeros((shifted.shape[0], (2*offset)))
for ts in (key_press_epochs.data * 1024)[:,0]:
    ep_power += shifted[:, int(ts - offset): int(ts + offset)]
ep_avg = ep_power / key_press_epochs.n_intervals

In [152]:
fig = plt.Figure()
ax = plt.gca()
pc_cwt = ax.pcolormesh(np.arange(0,2048,1), f_cwt, ep_avg)
pc_cwt.set_edgecolor('face')
cbar_cwt = fig.colorbar(pc_cwt, ax=ax)
cbar_cwt.ax.tick_params(labelsize=16)
cbar_cwt.set_label("Normalized PSD", fontsize=16, labelpad=15)
ax.set_title("Averaged Key Press Spectrogram (CWT) in Anterior Insula", fontsize=24)
ax.text(-0.25, 1.1, 'c', transform=ax.transAxes, size=27, weight='bold')
ax.set_xticks([0,1024,2048], labels=['-1','0','1'])
ax.set_xlabel('Time from Key Press (s)')
ax.set_ylabel('Frequency band (Hz)')

Text(0, 0.5, 'Frequency band (Hz)')